In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import requests
from bs4 import BeautifulSoup
from database import Scripts, get_db
from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright
from urllib.parse import urljoin
from sqlalchemy import select
from sqlalchemy.orm import selectinload


In [3]:
async with get_db() as db:
    script = (await db.execute(select(Scripts).options(selectinload(Scripts.script_details)))).scalars().first()

    print(script.script_details)


<ScriptDetails(Script=ACLBSL, listing_date=Jan 1, 2019, last_traded_price=1230.0, total_traded_quantity=6353, total_trades=95, previous_day_close_price=1245.0, high_price_low_price=1,269.00 - 1,224.00, week_52_high_low=1,491.00 - 585.00, open_price=1250.0, close_price=0.0, total_listed_shares=3671435, total_paid_up_value=367143488.0, market_capitalization=None)>


In [4]:
url = urljoin("https://nepalstock.com/",script.href)

In [5]:
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.set_extra_http_headers({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"})
    await page.goto(url)
    await page.wait_for_selector("table.table")
    table_data = {"script_id": script.id}
    table = page.locator("table.table").nth(0)
    rows = await table.locator("tr").all()
    for row in rows:
        td = await row.locator("td").all_text_contents()
        th = await row.locator("th").all_text_contents()
        if td and th:
            table_data[th[0].strip().replace("/","-")] = td[0].strip().replace("/","-")
    await browser.close()

In [6]:
table_data

{'script_id': 1,
 'Instrument Type': 'Equity ( EQ )',
 'Listing Date': 'Jan 1, 2019',
 'Last Traded Price': '1,097.00 \xa0 49.00 (4.68%)',
 'Total Traded Quantity': '4,730',
 'Total Trades': '92',
 'Previous Day Close Price': '1,048.00',
 'High Price - Low Price': '1,097.60 - 1,050.00',
 '52 Week High - 52 Week Low': '1,491.00 - 690.90',
 'Open Price': '1,055.00',
 'Close Price*': '1,089.65',
 'Total Listed Shares': '3,671,435',
 'Total Paid up Value': '367,143,488.00',
 'Market Capitalization': '4,000,579,147.75'}

In [7]:
from pydantic import BaseModel, Field, field_validator
from typing import Any, Optional

class StockDetails(BaseModel):
    script_id: Any
    instrument_type: str = Field(..., alias="Instrument Type")
    listing_date: str = Field(..., alias="Listing Date")
    last_traded_price: float = Field(..., alias="Last Traded Price")
    total_traded_quantity: int = Field(..., alias="Total Traded Quantity")
    total_trades: int = Field(..., alias="Total Trades")
    previous_day_close_price: float = Field(..., alias="Previous Day Close Price")
    high_price_low_price: str = Field(..., alias="High Price - Low Price")
    week_52_high_low: str = Field(..., alias="52 Week High - 52 Week Low")
    open_price: float = Field(..., alias="Open Price")
    close_price: float = Field(..., alias="Close Price*")
    total_listed_shares: int = Field(..., alias="Total Listed Shares")
    total_paid_up_value: float = Field(..., alias="Total Paid up Value")
    market_capitalization: float = Field(..., alias="Market Capitalization")

    @field_validator("total_listed_shares","last_traded_price", mode="after")
    def check_value(cls, value):
        if value <= 0:
            raise ValueError("Value should be greater than 0")
        return value

    @field_validator("last_traded_price", mode="before")
    def split_differences(cls, value):
        return float(value.split()[0]) if isinstance(value, str) else value

    @field_validator("total_traded_quantity","total_trades","previous_day_close_price","open_price","close_price","total_listed_shares","total_paid_up_value", "market_capitalization", mode="before")
    def remove_commas(cls, value):
        return float(value.replace(",", "")) if isinstance(value, str) else value

    class Config:
        populate_by_name = True
try:
    stock_details = StockDetails.model_validate(table_data)
except ValueError as e:
    print(e)

print(stock_details)


1 validation error for StockDetails
Last Traded Price
  Value error, could not convert string to float: '1,097.00' [type=value_error, input_value='1,097.00 \xa0 49.00 (4.68%)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


NameError: name 'stock_details' is not defined

In [ ]:
stock_details.model_dump()

{'script_id': 1,
 'instrument_type': 'Equity ( EQ )',
 'listing_date': 'Nov 17, 2009',
 'last_traded_price': 263.3,
 'total_traded_quantity': 98743,
 'total_trades': 375,
 'previous_day_close_price': 262.5,
 'high_price_low_price': '268.20 - 262.00',
 'week_52_high_low': '349.00 - 149.00',
 'open_price': 262.5,
 'close_price': 263.3,
 'total_listed_shares': 38480030,
 'total_paid_up_value': 3848003000.0,
 'market_capitalization': 10131791899.0}

In [ ]:
from database import ScriptDetails


async with get_db() as db:
    db.add(ScriptDetails(**stock_details.model_dump()))
    db.commit()

DetachedInstanceError: Parent instance <Scripts at 0x770b3b7fbfb0> is not bound to a Session; lazy load operation of attribute 'script_details' cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)